In [19]:

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the webdriver
driver = webdriver.Chrome()

# Visit the website
url = "https://tes.collegesource.com/publicview/TES_publicview01.aspx?rid=f080a477-bff8-46df-a5b2-25e9affdd4ed&aid=27b576bb-cd07-4e57-84d0-37475fde70ce"
driver.get(url)

# Wait for 2 seconds
import time
time.sleep(2)

# Get the page source
page_source = driver.page_source

# Parse the page source with BeautifulSoup

# Find the table element with id "gdvInstWithEQ"
table = driver.find_element(By.XPATH, '//*[@id="gdvInstWithEQ"]')

# print the table rows
# Loop through rows 0-49 to find elements with pattern gdvInstWithEQ_btnCreditFromInstName_{i}

data = {}
for i in range(50):
    try:
        # Find element by ID
        uni_elm = driver.find_element(By.ID, f"gdvInstWithEQ_btnCreditFromInstName_{i}")
        # Print the text and click the element
        uni_name = uni_elm.text
        time.sleep(1)
        uni_elm.click()
        # wait 2 seconds for the page to load
        time.sleep(5)

        
    

         
        # loop through until there is no element found
        courses = []
        counter = 0
        # Check if pagination info exists
        try:
            pagination_info = driver.find_element(By.ID, "lblCourseEQPaginationInfo")
            print(f"Pagination info found: {pagination_info.text}")
        except:
            print("No pagination info found")
        for i in range(50):
            counter += 1
            try:
                alt_course_name = driver.find_element(By.ID, f"gdvCourseEQ_lblReceiveCourseCode_{counter}")
              
                rpi_course_name = driver.find_element(By.ID, f"gdvCourseEQ_btnViewCourseEQDetail_{counter}")

                courses_data = {
                    "alt_course_name": alt_course_name.text,
                    "rpi_course_name": rpi_course_name.text
                }
                courses.append(courses_data)
                # note oth of tehse down
            except:
                break
        print(courses)
                
    except:
        # Skip if element not found 
        continue





No pagination info found
[{'alt_course_name': 'LITR 2000 LITERATURE ELECTIVE ()', 'rpi_course_name': 'ARH260 FROM DISNEY TO BURTON AND BEYOND: THE HIS OF ANIMATION (3)'}, {'alt_course_name': 'BIOL 2000 BIOLOGY ELECTIVE ()', 'rpi_course_name': 'BIO204 HUMAN ANATOMY AND PHYSIOLOGY II (4)'}, {'alt_course_name': 'BIOL 2000 BIOLOGY ELECTIVE ()', 'rpi_course_name': 'BIO208 MICROBIOLOGY (3)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CHE181 SAFETY IN LABORATORY AND STUDIO (1)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CSC156 DISCRETE STRUCTURES (3)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CSC170 INTRO TO COMPS AND THEIR APPLICATIONS (3)'}, {'alt_course_name': 'CSCI 1100 COMPUTER SCIENCE I (4)', 'rpi_course_name': 'CSC171 INTRO TO COMP PROGRAMMING (LECTURE) (4)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CSC190 COMP SCI ORIENTATION SEMINAR (1)'}, {'alt_course_name': 'ECON 1000 ECONOMICS ELECTIVE ()', 'r

In [35]:
def scrape_class_data(driver):
    # loop through until there is no element found
    courses = []
    counter = 0
    # Check if pagination info exists
    for i in range(50):
        counter += 1
        try:
            alt_course_name = driver.find_element(By.ID, f"gdvCourseEQ_lblReceiveCourseCode_{counter}")
            
            rpi_course_name = driver.find_element(By.ID, f"gdvCourseEQ_btnViewCourseEQDetail_{counter}")

            courses_data = {
                "alt_course_name": alt_course_name.text,
                "rpi_course_name": rpi_course_name.text
            }
            courses.append(courses_data)
            # note oth of tehse down
        except:
            break
    print(courses)
    return courses


In [ ]:
def scrape_uni_page(driver):
    # get the first page
    all_courses = scrape_class_data(driver)

    # number of pages
    pagination_table = driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
    page_cells = pagination_table.find_elements(By.TAG_NAME, "td")
    num_pages = len(page_cells)

    for page in range(2, num_pages +1):
        # Need to refind the pagination table and link each time since page reloads
        pagination_table = driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
        page_link = pagination_table.find_element(By.LINK_TEXT, str(page))
        page_link.click()
        time.sleep(1)
        # get the courses and extend
        curr_page_courses = scrape_class_data(driver)
        all_courses.extend(curr_page_courses)

    return all_courses


In [3]:

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the webdriver
driver = webdriver.Chrome()

# Visit the website
url = "https://tes.collegesource.com/publicview/TES_publicview01.aspx?rid=f080a477-bff8-46df-a5b2-25e9affdd4ed&aid=27b576bb-cd07-4e57-84d0-37475fde70ce"
driver.get(url)

# Wait for 2 seconds
import time
time.sleep(2)
# Get the page source
page_source = driver.page_source

# Find the pagination table
pagination_table = driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
page_cells = pagination_table.find_elements(By.TAG_NAME, "td")
# Get total number of pages
last_cell = page_cells[-1]
total_pages = int(last_cell.find_element(By.TAG_NAME, "a").text)

# Function to process a single page
def process_page(page_num, url):
    # Create new driver for this thread
    thread_driver = webdriver.Chrome()
    
    try:
        # Navigate to URL
        thread_driver.get(url)
        time.sleep(2)
        
        # If page number > 10, need to click dots first
        if page_num > 10:
            dots = thread_driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.LINK_TEXT, "...")
            dots.click()
            time.sleep(1)
        
        # Find and click the page number
        pagination_table = thread_driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
        page_link = pagination_table.find_element(By.LINK_TEXT, str(page_num))
        page_link.click()
        time.sleep(1)
        
        # Process page content here
        # ...
        
    finally:
        thread_driver.quit()

# Create and start threads for each page
import threading
threads = []
for page_num in range(1, total_pages + 1):
    thread = threading.Thread(target=process_page, args=(page_num, url))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()




